In [1]:
import os
os.environ.get('TMPDIR')

'/local/tmp.2275118'

In [2]:
from datasets import load_dataset

dataset = load_dataset("ASSERT-KTH/DISL", 'decomposed', cache_dir=os.environ.get('TMPDIR'), num_proc=16)['train']


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [3]:
index = list(range(len(dataset)))
dataset_idx = dataset.add_column('index', index)
dataset_idx

Dataset({
    features: ['contract_name', 'file_path', 'contract_address', 'language', 'source_code', 'abi', 'compiler_version', 'optimization_used', 'runs', 'constructor_arguments', 'evm_version', 'library', 'license_type', 'proxy', 'implementation', 'swarm_source', 'index'],
    num_rows: 514506
})

In [4]:
# remove Vyper files (check language column)
dataset_sol = dataset_idx.filter(lambda x: x['language'] != 'Vyper')
dataset_sol



Dataset({
    features: ['contract_name', 'file_path', 'contract_address', 'language', 'source_code', 'abi', 'compiler_version', 'optimization_used', 'runs', 'constructor_arguments', 'evm_version', 'library', 'license_type', 'proxy', 'implementation', 'swarm_source', 'index'],
    num_rows: 513639
})

In [5]:
dataset_min = dataset_sol.remove_columns(['contract_name', 'file_path', 'contract_address', 'language', 'abi', 'compiler_version', 'optimization_used', 'runs', 'constructor_arguments', 'evm_version', 'library', 'license_type', 'proxy', 'implementation', 'swarm_source'])
dataset_min

Dataset({
    features: ['source_code', 'index'],
    num_rows: 513639
})

# Create labels for the dataset
For every source code call the change_require_statements_string funtion, this function will return a list of source codes with the require statements removed
The original source code will be the label ground truth and the source code without the require statements will be the input


In [6]:
import sys
from feature_extraction.require2token import change_require_statements_string

def create_labels(batch):
    infilled_batch = []
    source_codes = batch['source_code']
    indices = batch['index']
    
    for i in range(len(source_codes)):
        infillings = change_require_statements_string(source_codes[i], "require(<FILL_ME>)")
        for infill in infillings:
            infilled_batch.append({'input': infill[0], 'label': infill[1], 'original_idx': indices[i], 'predicate': infill[2], 'comment': infill[3]})
    
    return {'data': infilled_batch }

sys.path.insert(0, '..')

dataset_labels = dataset_min.map(
    create_labels,
    batched=True,
    num_proc=16,
    batch_size=1000,
    remove_columns=['source_code', 'index']
)
dataset_labels

Map (num_proc=16):   0%|          | 0/513639 [00:00<?, ? examples/s]

line 108:15 extraneous input '=' expecting {'from', '{', '}', '(', 'error', 'for', 'function', 'address', 'calldata', 'if', 'return', 'revert', 'byte', 'let', 'switch', 'callback', BooleanLiteral, DecimalNumber, HexNumber, HexLiteralFragment, 'break', 'continue', 'leave', 'payable', 'global', 'constructor', 'receive', Identifier, StringLiteralFragment}

line 116:15 extraneous input '=' expecting {<EOF>, 'pragma', '~', 'import', 'from', '{', '}', 'abstract', 'contract', 'interface', 'library', '(', 'error', 'using', 'for', '+', '-', 'struct', 'modifier', 'function', 'event', 'enum', '[', 'address', 'mapping', 'calldata', 'if', 'try', 'while', 'unchecked', 'assembly', 'do', 'return', 'throw', 'emit', 'revert', 'var', 'bool', 'string', 'byte', '++', '--', 'new', 'delete', '!', 'let', 'switch', 'callback', Int, Uint, Byte, Fixed, Ufixed, BooleanLiteral, DecimalNumber, HexNumber, HexLiteralFragment, 'break', 'continue', 'leave', 'payable', 'type', 'global', 'constructor', 'fallback', 'recei

_pairs[from]?require(to==owner()||to==_launcher,"PEEN: trading disabled"):require(from==owner()||from==_launcher,"PEEN: trading disabled");


line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.6' expecting 'constant'
line 41:4 no viable alternative at input '_}'
line 48:4 no viable alternative at input '_}'
line 236:4 no viable alternative at input '_}'


_requireDeployer();


...

_requireSenderIsTimelock();
_requireSenderIsPendingTimelock();
_requireSenderIsTimelock();
_requireSenderIsPendingTimelock();
_requireNotPaused();
_requirePaused();


line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.5' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.0' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.4' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.6' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.9' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.3' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.3' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.4' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.6' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.0' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.9

_requireNotPaused();
_requirePaused();


line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.8' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.1' expecting 'constant'
line 1:12 mismatched input '.0' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.3' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.8' expecting 'constant'


Dataset({
    features: ['data'],
    num_rows: 5285991
})

In [7]:
dataset_flat = dataset_labels.flatten().rename_columns({'data.input': 'input', 'data.label': 'label', 'data.original_idx': 'original_idx', 'data.predicate': 'predicate', 'data.comment': 'comment'})
dataset_flat

Dataset({
    features: ['comment', 'input', 'label', 'original_idx', 'predicate'],
    num_rows: 5285991
})

In [8]:
dataset_flat.save_to_disk('/mimer/NOBACKUP/groups/naiss2024-23-121/morello/dataset2')

Saving the dataset (0/255 shards):   0%|          | 0/5285991 [00:00<?, ? examples/s]

In [9]:
# dataset_orig.push_to_hub('ASSERT-KTH/DISL', data_dir ='data/require', token='TOKEN')